In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import collections
import matplotlib.pyplot as plt
import keras.layers as KL
import keras.optimizers as KO
import keras.models as KM
import keras.backend as K

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


sample_submission.csv
test.tsv
train.tsv



In [2]:
train_raw = pd.read_csv('../input/train.tsv', sep='\t')
test_raw = pd.read_csv('../input/test.tsv', sep='\t')

In [3]:
print(test_raw.keys())

Index(['test_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping', 'item_description'],
      dtype='object')


In [4]:
brand_list = []
brand_list.extend(train_raw['brand_name'])
brand_list.extend(test_raw['brand_name'])
brand_list = [i for i in brand_list if str(i) != 'nan']
brand_collection = collections.Counter(brand_list)
max_brand_idx = list(brand_collection.values()).index(max(brand_collection.values()))
max_brand_name = list(brand_collection.keys())[max_brand_idx]
brand_dict = {'unknown': np.zeros(101)}
for i in range(len(train_raw)):
    price_idx = int(train_raw['price'][i] // 20)
    if str(train_raw['brand_name'][i]) == 'nan':
        brand_dict['unknown'][price_idx] = brand_dict['unknown'][price_idx] + 1
    else:
        brand_name = train_raw['brand_name'][i]
        if brand_name not in brand_dict:
            brand_dict[brand_name] = np.zeros(101)
        brand_dict[brand_name][price_idx] = brand_dict[brand_name][price_idx] + 1
print('brand dict size: ', len(brand_dict))
for k in brand_dict:
    brand_dict[k] = brand_dict[k] / sum(brand_dict[k])
print(list(brand_dict.values())[0])

brand dict size:  4810
[  6.55063681e-01   2.39388824e-01   6.04932652e-02   2.05806392e-02
   8.66153929e-03   5.18744014e-03   3.13901771e-03   2.11954821e-03
   1.24707199e-03   8.06092160e-04   8.70895647e-04   4.97880452e-04
   4.45721547e-04   2.10216191e-04   1.67540723e-04   2.54472231e-04
   9.64149446e-05   1.61218432e-04   7.11257788e-05   4.58366130e-05
   9.48343718e-05   3.95143216e-05   4.89977587e-05   1.73863015e-05
   1.10640100e-05   4.58366130e-05   9.48343718e-06   2.37085929e-05
   1.58057286e-05   4.74171859e-06   2.37085929e-05   9.48343718e-06
   1.10640100e-05   7.90286431e-06   6.32229145e-06   2.05474472e-05
   4.74171859e-06   7.90286431e-06   9.48343718e-06   1.58057286e-06
   1.26445829e-05   3.16114573e-06   3.16114573e-06   0.00000000e+00
   1.58057286e-06   1.10640100e-05   0.00000000e+00   6.32229145e-06
   0.00000000e+00   0.00000000e+00   9.48343718e-06   0.00000000e+00
   3.16114573e-06   1.58057286e-06   0.00000000e+00   3.16114573e-06
   0.000000

In [5]:
total_item_condition = []
total_item_condition.extend(train_raw['item_condition_id'])
total_item_condition.extend(test_raw['item_condition_id'])
print('total item condition id count: ', len(total_item_condition))
print(list(set(total_item_condition)))
condition_list = list(set(total_item_condition))
condition_dict = {}
for i in range(len(condition_list)):
    condition_dict[condition_list[i]] = np.zeros(5)
    condition_dict[condition_list[i]][i] = 1
print('condition dict: ', condition_dict)

total item condition id count:  2175894
[1, 2, 3, 4, 5]
condition dict:  {1: array([ 1.,  0.,  0.,  0.,  0.]), 2: array([ 0.,  1.,  0.,  0.,  0.]), 3: array([ 0.,  0.,  1.,  0.,  0.]), 4: array([ 0.,  0.,  0.,  1.,  0.]), 5: array([ 0.,  0.,  0.,  0.,  1.])}


In [6]:
price_list = train_raw['price']
print('min price: ', min(price_list))
print('max price: ', max(price_list))
print('intervals: ', 2020//20)
price_seg = []
for p in train_raw['price']:
    row = np.zeros(101)
    row[int(p//20)] = 1
    price_seg.append(row)
price_seg = np.array(price_seg)
print('price seg shape: ', price_seg.shape)

min price:  0.0
max price:  2009.0
intervals:  101
price seg shape:  (1482535, 101)


In [7]:
min_category_cnt = 10
max_category_cnt = 0
max_len = 0
category_list = []
category_list.extend(train_raw['category_name'])
category_list.extend(test_raw['category_name'])
category_list = [c for c in category_list if str(c) != 'nan']
print(category_list[100].lower())
label_list = []
char_list = []
for c in category_list:
    clist = c.split('/')
    char_list.extend(list(c))
    label_list.extend(clist)
    max_len = max(max_len, len(c))
    min_category_cnt = min(min_category_cnt, len(clist))
    max_category_cnt = max(max_category_cnt, len(clist))
print('min: ', min_category_cnt,  'max: ', max_category_cnt)
print('total labels: ', len(label_list))
print('max length: ', max_len)

women/tops & blouses/t-shirts
min:  3 max:  5
total labels:  6510440
max length:  65


In [8]:
category_dict = {'unknown': np.zeros(101)}
for i in range(len(train_raw)):
    price_idx = int(train_raw['price'][i] // 20)
    if str(train_raw['category_name'][i]) == 'nan':
        category_dict['unknown'][price_idx] = category_dict['unknown'][price_idx] + 1
    else:
        clist = train_raw['category_name'][i].split('/')
        cuname = ''
        for cn in clist:
            cuname = cuname + cn
            if cuname not in category_dict:
                category_dict[cuname] = np.zeros(101)
            category_dict[cuname][price_idx] = category_dict[cuname][price_idx] + 1
print('dict size: ', len(category_dict))

dict size:  1443


In [9]:
for k in category_dict:
    category_dict[k] = category_dict[k]/sum(category_dict[k])
print(list(category_dict.values())[0])

[  5.69938359e-01   2.91291291e-01   7.33364944e-02   2.57626047e-02
   1.28022760e-02   6.95432274e-03   5.21574206e-03   3.95131974e-03
   2.37079184e-03   1.26442232e-03   2.84495021e-03   9.48316738e-04
   6.32211159e-04   3.16105579e-04   4.74158369e-04   3.16105579e-04
   3.16105579e-04   4.74158369e-04   0.00000000e+00   0.00000000e+00
   1.58052790e-04   0.00000000e+00   1.58052790e-04   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.58052790e-04   0.00000000e+00   1.58052790e-04   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.58052790e-04   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00

In [10]:
unknown_cnt = 0
for i in range(len(test_raw)):
    if str(test_raw['category_name'][i]) == 'nan':
        unknown_cnt = unknown_cnt + 1
    else:
        clist = test_raw['category_name'][i].split('/')
        cuname = ''
        found = False
        for cn in clist:
            cuname = cuname + cn
            if cuname in category_dict:
                found = True
        if not found:
            unknown_cnt = unknown_cnt + 1
print('unknown rate: ', unknown_cnt/len(test_raw))

unknown rate:  0.0044104136529561164


In [11]:
X = []
y = np.array([[p] for p in train_raw['price']])
for i in range(len(train_raw)):
    row = None
    brand_name = 'unknown'
    if str(train_raw['brand_name'][i]) in brand_dict:
        brand_name = train_raw['brand_name'][i]
    row = brand_dict[brand_name]
    category_features = category_dict['unknown']
    if str(train_raw['category_name'][i]) != 'nan':
        clist = train_raw['category_name'][i].split('/')
        cuname = ''
        for cn in clist:
            cuname = cuname + cn
            if cuname in category_dict:
                category_features = category_dict[cuname]
    row = np.hstack([row, category_features])
    condition_features = condition_dict[train_raw['item_condition_id'][i]]
    row = np.hstack([row, condition_features])
    shipping_features = np.array([train_raw['shipping'][i]])
    row = np.hstack([row, shipping_features])
    X.append(row)
X = np.array(X)
print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (1482535, 208)
y shape:  (1482535, 1)


In [12]:
X_test = []
for i in range(len(test_raw)):
    row = None
    brand_name = 'unknown'
    if str(test_raw['brand_name'][i]) in brand_dict:
        brand_name = test_raw['brand_name'][i]
    row = brand_dict[brand_name]
    category_features = category_dict['unknown']
    if str(test_raw['category_name'][i]) != 'nan':
        clist = test_raw['category_name'][i].split('/')
        cuname = ''
        for cn in clist:
            cuname = cuname + cn
            if cuname in category_dict:
                category_features = category_dict[cuname]
    row = np.hstack([row, category_features])
    condition_features = condition_dict[test_raw['item_condition_id'][i]]
    row = np.hstack([row, condition_features])
    shipping_features = np.array([test_raw['shipping'][i]])
    row = np.hstack([row, shipping_features])
    X_test.append(row)
X_test = np.array(X_test)
print('X_test shape: ', X_test.shape)

X_test shape:  (693359, 208)


In [13]:
print(X_test[0])

[  6.55063681e-01   2.39388824e-01   6.04932652e-02   2.05806392e-02
   8.66153929e-03   5.18744014e-03   3.13901771e-03   2.11954821e-03
   1.24707199e-03   8.06092160e-04   8.70895647e-04   4.97880452e-04
   4.45721547e-04   2.10216191e-04   1.67540723e-04   2.54472231e-04
   9.64149446e-05   1.61218432e-04   7.11257788e-05   4.58366130e-05
   9.48343718e-05   3.95143216e-05   4.89977587e-05   1.73863015e-05
   1.10640100e-05   4.58366130e-05   9.48343718e-06   2.37085929e-05
   1.58057286e-05   4.74171859e-06   2.37085929e-05   9.48343718e-06
   1.10640100e-05   7.90286431e-06   6.32229145e-06   2.05474472e-05
   4.74171859e-06   7.90286431e-06   9.48343718e-06   1.58057286e-06
   1.26445829e-05   3.16114573e-06   3.16114573e-06   0.00000000e+00
   1.58057286e-06   1.10640100e-05   0.00000000e+00   6.32229145e-06
   0.00000000e+00   0.00000000e+00   9.48343718e-06   0.00000000e+00
   3.16114573e-06   1.58057286e-06   0.00000000e+00   3.16114573e-06
   0.00000000e+00   0.00000000e+00

In [14]:
model = None
if 'saved_model.h5' in set(check_output(["ls"]).decode("utf8").split('\n')):
    model = KM.load_model('saved_model.h5')
else:
    model = KM.Sequential()
    model.add(KL.Dense(512, activation='relu', input_dim=208))
    model.add(KL.Dense(256, activation='relu'))
    model.add(KL.Dense(128, activation='relu'))
    model.add(KL.Dropout(0.2))
    model.add(KL.Dense(1, activation='relu'))
    model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [ ]:
H = model.fit(x=X, y=y, batch_size=32, verbose=1, epochs=1)

Epoch 1/1
  15424/1482535 [..............................] - ETA: 4:31 - loss: 1013.4251

In [20]:
pred = model.predict(x=X_test, verbose=1)
print('pred shape: ', pred.shape, 'should be: ', len(test_raw))
predictions = []
for i in range(pred.shape[0]):
    predictions.append(pred[i,0])
output_dict = {'test_id':test_raw['test_id'],'price':predictions}
output = pd.DataFrame(output_dict)
print('output length: ', len(output))
output.to_csv('submission.csv', index=False)
model.save('saved_model.h5')
print(check_output(["ls"]).decode("utf8"))

693359/693359 [==============================] - 31s 45us/step
pred shape:  (693359, 1) should be:  693359
output length:  693359
__notebook_source__.ipynb
saved_model.h5
submission.csv

